In [2]:
def get_precision(y_pred, y_true):
    ## YOUR CODE HERE...
    n = len(y_pred)
    fps,tps = 0,0
    for i in range(n):
        if y_pred[i] > y_true[i]:
            fps+=1
            continue;
#         if y_pred[i] < y_true[i]:
#             fns+=1
#             continue
#         if (y_pred[i] + y_true[i]) == 0:
#             tns+=1
#             continue
        if (y_pred[i] + y_true[i]) > 1:
            tps+=1
            continue
    precision = tps/(tps + fps)
    return precision

def get_recall(y_pred, y_true):
    ## YOUR CODE HERE...
    n = len(y_pred)
    tps, fns = 0,0
    for i in range(n):
#         if y_pred[i] > y_true[i]:
#             fps+=1
#             continue;
        if y_pred[i] < y_true[i]:
            fns+=1
            continue
#         if (y_pred[i] + y_true[i]) == 0:
#             tns+=1
#             continue
        if (y_pred[i] + y_true[i]) > 1:
            tps+=1
            continue
    recall = tps/(fns + tps)
    return recall

def get_fscore(y_pred, y_true):
    ## YOUR CODE HERE...
    recall = get_recall(y_pred, y_true)
    precision = get_precision(y_pred, y_true)
    fscore = 2/(1/precision + 1/recall)
    return fscore

In [2]:
y_pred = [1,0,1,0,1,0,1,0]
y_true = [0,1,0,0,1,1,1,1]

In [3]:
print(get_recall(y_pred,y_true))
print(get_precision(y_pred,y_true))
print(get_fscore(y_pred,y_true))

0.4
0.5
0.4444444444444444


In [1]:
def load_file(data_file, sure):
    words = []
    labels = []
    with open(data_file,'rt',encoding="utf8") as f:
        i = 0
        for line in f:
            if i > 0:
                line_split = line[:-1].split("\t")
                words.append(line_split[0].lower())
                if sure:
                    labels.append(int(line_split[1]))
            i += 1
    return words,labels

In [3]:
import numpy as np
import gzip
from collections import defaultdict
from statistics import median
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from nltk.corpus import wordnet
import nltk
import syllables
from sklearn import svm

In [4]:
def all_complex(datafile):
    words, labels = load_file(datafile)
    #preds = np.ones_like(labels)
    preds = [1]*len(labels)
    labels = [1 if i == 0 else 0 for i in labels]
    precision = get_precision(preds,labels)
    recall = get_recall(preds, labels)
    fscore = get_fscore(preds, labels)
    performance = [precision,recall,fscore]
    return performance
            

In [5]:
print(all_complex("data/complex_words_development.txt"))

TypeError: load_file() missing 1 required positional argument: 'sure'

In [8]:
# function for labeling word length 
def length_threshold_feature(words, threshold):
    return [1 if len(i)>= threshold else 0 for i in words]

In [9]:
def word_length_threshold(training_file, development_file):
    # t stands for training_file, d stands for development_file
    w_t, l_t = load_file(training_file)
    w_d, l_d = load_file(development_file)
    # find the max fscore during the iteration
    fscore_max_t = 0
#     fscore_max_d = 0
    precision_t_max = 0
#     precision_d_max = 0
    recall_t_max = 0
#     recall_d_max = 0
    threshold_t = 0
#     threshold_d = 0
    for i in range(10):
        pred_t = length_threshold_feature(w_t,i)
        precision_t,recall_t,fscore_t = get_precision(pred_t,l_t),get_recall(pred_t,l_t),get_fscore(pred_t,l_t)
        if fscore_t > fscore_max_t:
            fscore_max_t = fscore_t
            precision_t_max = precision_t
            recall_t_max = recall_t
            threshold_t = i
#         if fscore_d > fscore_max_d:
#             fscore_max_d = fscore_d
#             precision_d_max = precision_d
#             recall_d_max = recall_d
#             threshold_d = i
#         print("Threshold :%d ,precision in training: %f ,recall in t: %f,fcore in t: %f"% (i,precision_t,recall_t,fscore_t))
#         print("Threshold :%d ,precision in dev: %f ,recall in dev: %f,fcore in dev: %f"% (i,precision_d,recall_d,fscore_d))
    pred_d = length_threshold_feature(w_d,threshold_t)
    precision_d,recall_d,fscore_d = get_precision(pred_d,l_d),get_recall(pred_d,l_d),get_fscore(pred_d,l_d)
    print("Best performance in training, threshold : %d, precision %f, recall %f, fscore %f "%(threshold_t,precision_t_max,recall_t_max,fscore_max_t))
    print("Best performance in development, threshold : %d, precision %f, recall %f, fscore %f "%(threshold_t,precision_d,recall_d,fscore_d))
    training_performance = [precision_t, recall_t, fscore_t]
    development_performance = [precision_d, recall_d, fscore_d]
    return training_performance, development_performance    

In [381]:
training_file = "data/complex_words_training.txt"
development_file = "data/complex_words_development.txt"
test_file = "data/complex_words_test_unlabeled.txt"
# print(word_length_threshold(training_file,development_file))

In [22]:
## load Google Ngram counts
def load_ngram_counts(ngram_counts_file):
    counts = defaultdict(int)
    with gzip.open(ngram_counts_file,'rt', encoding="utf8") as f:
        for line in f:
            token,count = line.strip().split('\t')
            if token[0].islower():
                counts[token] = int(count)
    return counts
    

In [466]:
def frequency_threshold_feature(words, threshold, counts):
    result = []
    for i in range(len(words)):
        wordFreq = counts.get(words[i].lower())
        if wordFreq is None:
            result.append(1)
        elif wordFreq <= threshold:
            result.append(1)
        elif wordFreq > threshold:
            result.append(0)
    return  result

In [465]:
def word_frequency_threshold(training_file, development_file, counts):
    
    twords, tlabels = load_file(training_file,True)
    dwords, dlabels = load_file(development_file,True)
    max_in_t = find_max_frequency(counts, twords)
    max_in_d = find_max_frequency(counts, dwords)
    min_in_t = find_min_frequency(counts, twords)
    fscore_max = 0
    freq = 0
#     for i in range(100000, 200000000 + 10,2000):
    preds = frequency_threshold_feature(twords,25000000, counts)
    fscore = get_fscore(preds, tlabels)
    fscore_max,freq = (fscore,25000000) if fscore > fscore_max else (fscore_max,freq)
    
    print("best freqency %d"% freq)
    dpred = frequency_threshold_feature(dwords, freq, counts)
    tpred = frequency_threshold_feature(twords, freq,counts)
    
    dprecision,drecall,dfscore = get_precision(dpred,dlabels),get_recall(dpred,dlabels),get_fscore(dpred,dlabels)
    tprecision,trecall,tfscore = get_precision(tpred,tlabels),get_recall(tpred,tlabels),get_fscore(tpred,tlabels)
    
    training_performance = [tprecision, trecall, tfscore]
    development_performance = [dprecision, drecall, dfscore]
    return training_performance, development_performance

In [14]:
def find_max_frequency(counts, words):
    max_freq = 0
    for word in words:
        if word in counts:
            freq = counts.get(word)
            max_freq = freq if freq > max_freq else max_freq
    return max_freq

In [434]:
find_min_frequency(counts, twords)

256

In [15]:
def find_min_frequency(counts, words):
    min_freq = 1000
    for word in words:
        if word in counts:
            freq = counts.get(word)
            min_freq = freq if freq < min_freq else min_freq
    return min_freq

In [114]:
ngram_counts_file = "ngram_counts.txt.gz"
counts = load_ngram_counts(ngram_counts_file)

In [467]:
print(word_frequency_threshold(training_file,development_file,counts))

best freqency 25000000
([0.5450073782587309, 0.8562596599690881, 0.6660655244965434], [0.5457227138643068, 0.8851674641148325, 0.6751824817518248])


In [17]:
for key,val in counts.items():
    if val == 1120679362:
        print(key)

time


In [431]:
def naive_bayes(training_file, development_file, counts):
    ## YOUR CODE HERE
    twords, tlabels = load_file(training_file,True)
    dwords, dlabels = load_file(development_file,True)
    tlength = [ 0 if len(word) == None else len(word)for word in twords]
    dlength = [ 0 if len(word) == None else len(word)for word in dwords]
    tfreq = [ 0 if counts.get(w) == None else counts.get(w) for w in twords]
    dfreq = [ 0 if counts.get(w) == None else counts.get(w) for w in dwords]
    
    tl = np.array(tlength)
    meanl = np.mean(tl)
    stdl  = np.std(tl)
    tl_scale = [(l - meanl)/stdl for l in tl]
    dl = np.array(dlength)
    dl_scale = [(l - meanl)/stdl for l in dl]
    tf = np.array(tfreq)
    meanf = np.mean(tf)
    stdf = np.std(tf)
    tf_scale = [(f - meanf)/stdf for f in tf]
    df = np.array(dfreq)
    df_scale = [(f - meanf)/stdf for f in df]
    X_train = np.matrix([tl_scale,tf_scale]).T
    X_test = np.matrix([dl_scale,df_scale]).T
    Y = tlabels
    
    clf = GaussianNB()
    clf.fit(X_train, Y)
    dpred = clf.predict(X_test)
    tpred = clf.predict(X_train)
    
    tprecision,trecall,tfscore = get_precision(tpred,tlabels),get_recall(tpred,tlabels),get_fscore(tpred,tlabels)
    dprecision,drecall,dfscore = get_precision(dpred,dlabels),get_recall(dpred,dlabels),get_fscore(dpred,dlabels)
    training_performance = (tprecision, trecall, tfscore)
    development_performance = (dprecision, drecall, dfscore)
    return training_performance,development_performance

In [432]:
print(naive_bayes(training_file,development_file,counts))

((0.4918351477449456, 0.9775888717156105, 0.6544231764097258), (0.4700352526439483, 0.9569377990430622, 0.6304176516942475))


In [20]:
def logistic_regression(training_file, development_file, counts):
    ## YOUR CODE HERE
    twords, tlabels = load_file(training_file)
    dwords, dlabels = load_file(development_file)
    
    tlength = [ 0 if len(word) == None else len(word)for word in twords]
    dlength = [ 0 if len(word) == None else len(word)for word in dwords]
    tfreq = [ 0 if counts.get(w) == None else counts.get(w) for w in twords]
    dfreq = [ 0 if counts.get(w) == None else counts.get(w) for w in dwords]
    
    tl = np.array(tlength)
    meanl = np.mean(tl)
    stdl  = np.std(tl)
    tl_scale = [(l - meanl)/stdl for l in tl]
    dl = np.array(dlength)
    dl_scale = [(l - meanl)/stdl for l in dl]
    tf = np.array(tfreq)
    meanf = np.mean(tf)
    stdf = np.std(tf)
    tf_scale = [(f - meanf)/stdf for f in tf]
    df = np.array(dfreq)
    df_scale = [(f - meanf)/stdf for f in df]
    X_train = np.matrix([tl_scale,tf_scale]).T
    X_test = np.matrix([dl_scale,df_scale]).T
    Y = tlabels
    
    lr = LogisticRegression(C=1000.0, random_state=0, solver='liblinear')
    lr.fit(X_train,Y)
    tpred = lr.predict(X_train)
    dpred = lr.predict(X_test)
    tprecision,trecall,tfscore = get_precision(tpred,tlabels),get_recall(tpred,tlabels),get_fscore(tpred,tlabels)
    dprecision,drecall,dfscore = get_precision(dpred,dlabels),get_recall(dpred,dlabels),get_fscore(dpred,dlabels)
    
    training_performance = (tprecision, trecall, tfscore)
    development_performance = (dprecision, drecall, dfscore)
    return development_performance, training_performance

In [21]:
print(logistic_regression(training_file, development_file, counts))

((0.7235142118863049, 0.6698564593301436, 0.6956521739130435), (0.7243478260869565, 0.6437403400309119, 0.6816693944353518))


In [96]:
def get_syns_nums(words):
    syns = []
    for word in words:
        counts = 0
        for each in wordnet.synsets(word):
            counts += len(each.lemma_names())
        syns.append(counts)
    return syns

In [101]:
def get_syllables(words):
    return [syllables.count_syllables(word) for word in words]

In [105]:
def get_senses(words):
    return [len(wordnet.synsets(word)) for word in words]

In [39]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/weihaoran/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [382]:
twords, tlabels = load_file(training_file)
dwords, dlabels = load_file(development_file)
total_words = twords + dwords
total_labels = tlabels + dlabels
tsyns = regularization(get_syns_nums(twords))
dsyns = regularization(get_syns_nums(dwords))
tsyll = regularization(get_syllables(twords))
dsyll = regularization(get_syllables(dwords))
tsens = regularization(get_senses(twords))
dsens = regularization(get_senses(dwords))
syns = regularization(get_syns_nums(total_words))
syll = regularization(get_syllables(total_words))
sens = regularization(get_senses(total_words))

In [109]:
def regularization(paras):
    data = np.array(paras)
    mean = np.mean(data)
    std = np.std(data)
    return [(each - mean)/std for each in data]

In [383]:
tlength = regularization([ 0 if len(word) == None else len(word)for word in twords])
dlength = regularization([ 0 if len(word) == None else len(word)for word in dwords])
tfreq = regularization([ 0 if counts.get(w) == None else counts.get(w) for w in twords])
dfreq = regularization([ 0 if counts.get(w) == None else counts.get(w) for w in dwords])
length = regularization([ 0 if len(word) == None else len(word)for word in total_words])
freq = regularization([ 0 if counts.get(w) == None else counts.get(w) for w in total_words])

In [385]:
X_train = np.matrix([tlength, tfreq, tsyns, tsyll, tsens]).T
X_test = np.matrix([dlength, dfreq, dsyns, dsyll, dsens]).T
X_total = np.matrix([length, freq, syns,syll,sens]).T

#### SVM

In [410]:
svc = svm.SVC(gamma='scale')

In [411]:
svc.fit(X_train, tlabels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [416]:
tpred = svc.predict(X_train)
dpred = svc.predict(X_test)
pred_svc = svc.predict(X_total)

In [418]:
tprecision,trecall,tfscore = get_precision(tpred,tlabels),get_recall(tpred,tlabels),get_fscore(tpred,tlabels)
dprecision,drecall,dfscore = get_precision(dpred,dlabels),get_recall(dpred,dlabels),get_fscore(dpred,dlabels)
total_fscore_svc = get_fscore(pred_svc, total_labels)
training_performance_svm = (tprecision, trecall, tfscore)
development_performance_svm = (dprecision, drecall, dfscore)


In [419]:
development_performance_svm,training_performance_svm, total_fscore_svc

((0.6666666666666666, 0.8755980861244019, 0.7569803516028957),
 (0.7019011406844107, 0.7132921174652241, 0.7075507857416635),
 0.7123446082682857)

### KNN

In [131]:
from sklearn.neighbors import KNeighborsClassifier

In [136]:
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(X_train,tlabels)
tpred_knn = knn.predict(X_train)
dpred_knn = knn.predict(X_test)

In [137]:
knn_dev = get_fscore(dpred_knn,dlabels)

In [138]:
knn_dev

0.5895627644569816

### Decision tree

In [139]:
from sklearn import tree

In [143]:
dtc = tree.DecisionTreeClassifier()
dtc.fit(X_train,tlabels)
tpred_dtc = dtc.predict(X_train)
dpred_dtc = dtc.predict(X_test)
t_dtc = get_fscore(tpred_dtc, tlabels)
d_dtc = get_fscore(dpred_dtc, dlabels)

In [144]:
d_dtc

0.4384

### Random Forest

In [201]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.model_selection import cross_validate

In [241]:
param_test1 = {'n_estimators':range(300,1100,100)}
gsearch1 = GridSearchCV(estimator = RandomForestClassifier(min_samples_split=100,
                                  min_samples_leaf=20,max_depth=3,max_features='sqrt' ,random_state=0), 
                       param_grid = param_test1, scoring='f1',cv=5)
gsearch1.fit(X_train,tlabels)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=20, min_samples_split=100,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': range(300, 1100, 100)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [326]:
gsearch1.best_estimator_,gsearch1.best_score_,gsearch1.best_params_

(RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=3, max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=20, min_samples_split=100,
             min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=None,
             oob_score=False, random_state=0, verbose=0, warm_start=False),
 0.711022734623366,
 {'n_estimators': 400})

In [280]:
param_test2 = {'max_depth':range(1,10,1)}
gsearch2 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 60, 
                                  min_samples_leaf=20,max_features='sqrt' ,oob_score=True, random_state=0),
   param_grid = param_test2, scoring='f1',iid=False, cv=5)
gsearch2.fit(X_train,tlabels)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=20, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=60, n_jobs=None,
            oob_score=True, random_state=0, verbose=0, warm_start=False),
       fit_params=None, iid=False, n_jobs=None,
       param_grid={'max_depth': range(1, 10)}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring='f1', verbose=0)

In [281]:
gsearch2.best_params_, gsearch2.best_score_

({'max_depth': 7}, 0.7322393485136026)

In [313]:
param_test3 = {'min_samples_split':range(2,10,1), 'min_samples_leaf':range(2,60,10)}
gsearch3 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 400, max_depth=7,
                                  max_features='sqrt' ,oob_score=True, random_state=0),
   param_grid = param_test3, scoring='f1',iid=False, cv=5)
gsearch3.fit(X_train,tlabels)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=None,
            oob_score=True, random_state=0, verbose=0, warm_start=False),
       fit_params=None, iid=False, n_jobs=None,
       param_grid={'min_samples_split': range(2, 10), 'min_samples_leaf': range(2, 60, 10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [322]:
gsearch3.best_estimator_, gsearch3.best_score_,gsearch3.best_params_

(RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=7, max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=2, min_samples_split=9,
             min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=None,
             oob_score=True, random_state=0, verbose=0, warm_start=False),
 0.7329541119715254,
 {'min_samples_leaf': 2, 'min_samples_split': 9})

In [323]:
 param_test4 = {'max_features':range(1,4,1)}
gsearch4 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 400, max_depth=7, min_samples_split=9,
                                  min_samples_leaf=2 ,oob_score=True, random_state=10),
   param_grid = param_test4, scoring='roc_auc',iid=False, cv=5)
gsearch4.fit(X_train,tlabels)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=9,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=None,
            oob_score=True, random_state=10, verbose=0, warm_start=False),
       fit_params=None, iid=False, n_jobs=None,
       param_grid={'max_features': range(1, 4)}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring='roc_auc', verbose=0)

In [325]:
gsearch4.best_estimator_,gsearch4.best_score_,gsearch4.best_params_

(RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=7, max_features=2, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=2, min_samples_split=9,
             min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=None,
             oob_score=True, random_state=10, verbose=0, warm_start=False),
 0.8413455568882584,
 {'max_features': 2})

In [ ]:
cross_validate

In [407]:
rfc = RandomForestClassifier(n_estimators=400, max_depth=3, random_state=0, max_features=3,oob_score=True)
rfc.fit(X_train,tlabels)
# tpred_rfc = rfc.predict(X_train)
# dpred_rfc = rfc.predict(X_test)
pred_rfc = rfc.predict(X_total)

In [408]:
s_rfc = get_fscore(pred_rfc, total_labels)

In [409]:
s_rfc

0.7117037037037036

In [379]:
t_rfc = get_fscore(tpred_rfc, tlabels)
d_rfc = get_fscore(dpred_rfc, dlabels)


In [380]:
d_rfc

0.7377245508982035

### Neural Network

In [195]:
import tensorflow as tf